#### Author : Gautam Badri

# 2) Convolutional Neural Network

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
train_data = datasets.FashionMNIST(
    root="data",
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [5]:
train_data.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [6]:
num_classes = len(train_data.classes)

In [7]:
num_classes

10

### 2A) Create simple NN. 


### 2B) Convolution layer 1 on x (DO NOT flatten) -> ReLU -> Convolution layer 2


### 2C) Create a single fully connected layer (you need to figure out input size)


### 2D) Do softmax before returning in the forward function.

In [8]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=(5, 5))
        self.feature_map1_dim = (28-5)+1 
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=(5, 5))
        self.relu2=nn.ReLU()
        self.flatten = nn.Flatten()
        self.feature_map2_dim = (self.feature_map1_dim-5)+1
        self.fc1 = nn.Linear(in_features=(50*self.feature_map2_dim*self.feature_map2_dim), out_features=10)
        self.logSoftmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        # x = nn.functional.normalize(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        output = self.logSoftmax(x)
        return output

### 2E) Customized loss function (not the library version)

In [9]:
def loss_fun(y_pred, y_actual):
#   print(y_actual.shape,y_pred.shape)
  v = -(y_actual * torch.log(y_pred+1e-10))
#   print(y_pred)
#   print(v)
  v = torch.sum(v)
  return v

In [10]:
def get_optim(model, lr=1e-6):
    optim = torch.optim.SGD(model.parameters(), lr = lr)
    return optim

### 2F) Train on training data set

In [11]:
def train_network(train_dataloder, model, optim, loss_fn, epochs):
    print('Training Model ...\n\n')
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data
            inputs , labels = inputs.to(device), labels.to(device)
            optim.zero_grad()
            outputs = model(inputs)
            tmp = torch.nn.functional.one_hot(labels, num_classes= 10)
            loss = loss_fn(outputs, tmp)
            loss.backward()
            optim.step()
            running_loss += loss.item()
            # return
        print("[Epoch : {}/{}] loss = ".format(epoch+1,epochs),running_loss)

In [12]:
model = CNN()
model = model.to(device)
optim = get_optim(model)
train_network(train_dataloader, model, optim, loss_fun, 5)

Training Model ...


[Epoch : 0/5] loss =  133618.14059448242
[Epoch : 1/5] loss =  123481.16239929199
[Epoch : 2/5] loss =  106475.13614273071
[Epoch : 3/5] loss =  84968.65009307861
[Epoch : 4/5] loss =  68907.70473098755


In [13]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [14]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 8.9 MB/s 


In [15]:
from torchmetrics import Precision, Recall, F1Score, Accuracy
from torchmetrics.classification import accuracy

### 2G) Test on the test data set - report accuracy, precision, recall and F1 scores.

In [16]:
def test_network(dataloader, model, loss_fun):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            tmp = torch.nn.functional.one_hot(y, num_classes= 10)
            pred = model(X)
            test_loss += loss_fun(pred, tmp).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss/= num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    accuracy1 = Accuracy().to(device)
    print('Accuracy :', accuracy1(pred,y))
    precision = Precision(average = 'macro', num_classes = 10).to(device)
    print('precision :', precision(pred,y))

    recall = Recall(average = 'macro', num_classes = 10).to(device)
    print('recall :', recall(pred,y))
    f1_score = F1Score(average = 'macro', num_classes = 10).to(device)
    print('f1_score :', f1_score(pred,y))
    return accuracy1,precision, recall, f1_score

In [17]:
test_network(test_dataloader,model, loss_fun)

Test Error: 
 Accuracy: 67.2%, Avg loss: 67.702223 

Accuracy : tensor(0.7188, device='cuda:0')
precision : tensor(0.6857, device='cuda:0')
recall : tensor(0.7583, device='cuda:0')
f1_score : tensor(0.7096, device='cuda:0')


(Accuracy(), Precision(), Recall(), F1Score())